In [22]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
training=pd.read_csv('D:\\New folder (2)\\train.csv').copy()
testing=pd.read_csv('D:\\New folder (2)\\test.csv').copy()

In [23]:
def count_nulls(dataset):
    return sum(dataset.isnull())

In [24]:
def get_name(my_list, position):
    return my_list[position][:4]

In [25]:
training['Updated_Names']=training['Name'].str.split(',').apply(get_name, position=1)
training.drop('Name', axis=1, inplace=True)

In [26]:
d=dict()
count=0
for i in set(training['Updated_Names']):
    d[i]=count/15
    count+=1
training['Names']=training['Updated_Names'].map(d)
training.drop('Updated_Names', axis=1, inplace=True)

In [27]:
training['sex']=training['Sex'].map({"female":0, "male":1})
training.drop('Sex', axis=1, inplace=True)

In [28]:
training['pclass']=training['Pclass']/3
training.drop('Pclass', axis=1, inplace=True)

In [29]:
training.loc[training['Embarked'].isnull(), 'Embarked']='S'

In [30]:
training['embarked']=training.Embarked.map({"S":0, "C":0.5, "Q":1})
training.drop('Embarked', inplace=True, axis=1)

In [31]:
training['fare']=training.Fare/training.Fare.max()
training.drop('Fare', inplace=True, axis=1)

In [32]:
training['parch']=training['Parch']/training['Parch'].max()
training.drop('Parch', inplace=True, axis=1)

In [33]:
training['sibsp']=training['SibSp']/training['SibSp'].max()
training.drop('SibSp', inplace=True, axis=1)

In [34]:
cabins=[]
for i in training.loc[training['Cabin'].notnull(), 'Cabin']:
    cabins.append(i[0])

In [35]:
set_cabins=set(cabins)

In [36]:
dictionary_cabin=dict()
count=0
for i in set_cabins:
    dictionary_cabin[i]=count/8
    count+=1
dictionary_cabin['N']=1
dictionary_cabin

{'F': 0.0,
 'A': 0.125,
 'C': 0.25,
 'D': 0.375,
 'B': 0.5,
 'T': 0.625,
 'E': 0.75,
 'G': 0.875,
 'N': 1}

In [37]:
training.loc[training.Cabin.isnull(), 'Cabin']='N'

In [38]:
def cabin_mapping(get_list, position):
    return get_list[position][0]
training['cabin']=training.Cabin.str.split(' ').apply(cabin_mapping, position=0)

In [39]:
training.drop('Cabin', axis=1, inplace=True)

In [40]:
training['cabins']=training.cabin.map(dictionary_cabin)
training.drop('cabin', inplace=True, axis=1)

In [41]:
def ticket_mapping(get_ticket, position):
    if get_ticket[position]=='LINE':
        return -1
    return int(get_ticket[position])
training['ticket']=training['Ticket'].str.split(' ').apply(ticket_mapping, position=-1)
training.drop('Ticket', inplace=True, axis=1)

In [42]:
training['tickets']=training['ticket']/training.ticket.max()
training.drop('ticket', inplace=True, axis=1)

In [44]:
training.loc[(training.Age.isnull())&(training['Survived']==0), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==0), 'Age'].median()

In [45]:
training.loc[(training.Age.isnull())&(training['Survived']==1), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==1), 'Age'].median()

In [46]:
training['age']=training.Age/80.0
training.drop('Age', inplace=True, axis=1)

In [47]:
training

,Survived,Names,sex,pclass,embarked,fare,parch,sibsp,cabins,tickets,age
0,1,0.133333,0,0.666667,0.0,0.050749,0.000000,0.125,1.0,0.073651,0.3625
1,0,0.533333,1,1.000000,0.0,0.015713,0.000000,0.000,1.0,0.000795,0.3625
2,0,0.533333,1,0.666667,0.0,0.050749,0.000000,0.000,1.0,0.080822,0.4875
3,0,0.133333,0,1.000000,0.0,0.041136,0.666667,0.000,1.0,0.112826,0.3625
4,0,0.533333,1,1.000000,0.0,0.013761,0.000000,0.000,1.0,0.126422,0.3125
...,...,...,...,...,...,...,...,...,...,...,...
663,1,0.933333,0,0.666667,0.0,0.020495,0.000000,0.000,1.0,0.004800,0.2125
664,0,0.533333,1,1.000000,1.0,0.015127,0.000000,0.000,1.0,0.120150,0.3625
665,1,0.533333,1,1.000000,0.0,0.110272,0.000000,0.000,1.0,0.000516,0.4000
666,0,0.933333,0,1.000000,0.0,0.019202,0.000000,0.000,1.0,0.002435,0.2750


In [48]:
values=training.values
from sklearn.model_selection import train_test_split
X=values[:, 1:]
Y=values[:, 0]
x_train, x_test, y_train, y_test=train_test_split(X, Y)

In [58]:
# Fit the Logistic Regression model
clf = LogisticRegression(n_jobs=-1, solver="saga", max_iter=20000, C=10, tol=0.000001, class_weight="balanced")
clf.fit(x_train, y_train)

# Make predictions on the test set
predictions = clf.predict(x_test)

In [59]:
# Create a DataFrame to store predictions
prediction_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})


In [60]:
# Save the predictions to a CSV file
prediction_df.to_csv('prediction.csv', index=False)


In [61]:
# Your previous code for data preprocessing and model training here...

# Fit the Logistic Regression model
clf = LogisticRegression(n_jobs=-1, solver="saga", max_iter=20000, C=10, tol=0.000001, class_weight="balanced")
clf.fit(x_train, y_train)

# Make predictions on the test set
predictions = clf.predict(x_test)

# Create a DataFrame to store predictions
prediction_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

# Save the predictions to a CSV file
prediction_df.to_csv('prediction.csv', index=False)
